In [1]:
import os,json
import spacy
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")

In [3]:
path_folder='/home/xhsun/Desktop/gitRepositories/Some-NER-models/data/CoNLL03/en_conll03/'

In [4]:
import random

# 根据句子长度比例随机添加punckt

- 如果句子长度小于5,则不添加
- 如果长度在6-10之间，随机添加1个
- 如果长度在10-15之间，随机添加两个
- 长度大于15的句子，随机添加3个

In [12]:
puncts=[',','.','!','?','!!','??','#','"']

In [17]:
def parsing_sentence(sentence_list,ent_list):
    sentence=' '.join(sentence_list)
    doc=nlp(sentence)
    result=[]
    for i,token in enumerate(doc):
        word=token.text
        deprel=token.dep_.lower()
        head=token.head.text
        pos_tag=token.pos_
        if head not in sentence_list:
            #出现了parsing错误的问题，通常是因为一个词被分成了两个
            if i>=len(ent_list):
                i=len(ent_list)-1
                
            if head+sentence_list[i] in sentence_list:
                head=head+sentence_list[i]
            elif sentence_list[i-1]+head in sentence_list:
                head=sentence_list[i-1]+head
            else:
                head=sentence_list[i]#实在找不到
        head_id=sentence_list.index(str(head))
        if deprel=='root':
            head_id=0
        else:
            head_id+=1
        if i>=len(ent_list):
            i=len(ent_list)-1
        result.append([word,pos_tag,head_id,deprel,ent_list[i]])
    return result

def process_data(path_folder,write_path):
    files_path=os.listdir(path_folder)
    print(files_path)
    insert_sentence_count=0
    for file_name in files_path:
        with open(os.path.join(path_folder,file_name)) as f:
            lines=f.readlines()
        sentences_and_entlabels=[([],[])]
        for line in lines:
            if line.strip() in ['',' ']:
                sentences_and_entlabels.append(([],[]))
            else:
                line_split=line.strip().split()
                assert len(line_split)==2
                word,entity_label=line_split
                sentences_and_entlabels[-1][0].append(word)
                sentences_and_entlabels[-1][1].append(entity_label)
                
        if sentences_and_entlabels[-1]==([],[]):
            del sentences_and_entlabels[-1]
            
        for example in sentences_and_entlabels:
            sentence_list,label_list=example
            insert_=False
            for _ in range(len(sentence_list)//5):
                point=puncts[random.randint(a=0,b=len(puncts)-1)]
                pos=random.randint(a=0,b=len(sentence_list)-1)
                if label_list[pos]!='O':
                    continue#只有在O的位置插入punckt才合理
                else:
                    sentence_list.insert(pos,point)
                    label_list.insert(pos,'O')
                    insert_=True
                    
            insert_sentence_count+=int(insert_)
        print(insert_sentence_count)            
        with open(os.path.join(write_path,file_name+".conllx"),'w') as f:
            parsing_error_count=0
            for example in tqdm(sentences_and_entlabels):
                sentences,entlabels=example
                assert len(sentences)==len(entlabels)
                
                parsing_result=parsing_sentence(sentence_list=sentences,ent_list=entlabels)
#                 if parsing_result==None or len(parsing_result)!=len(entlabels):
#                     #parsing出现错误
#                     parsing_error_count+=1
#                     continue
                    
                for i in range(len(parsing_result)):
                    word,pos_tag,head_id,deprel,ent=parsing_result[i]
                    #ent=entlabels[i]
                    lemma='_'
                    feats='_'
                    conllx_example=[str(i+1),word,lemma,pos_tag,pos_tag,feats,str(head_id),deprel,'_','_',ent]
                    f.write('\t'.join(conllx_example)+'\n')
                    
                f.write('\n')
            print("parsing error count : ",parsing_error_count,len(sentences_and_entlabels))

In [18]:
process_data(path_folder=path_folder,write_path='/home/xhsun/Desktop/gitRepositories/Some-NER-models/data/NoiseCoNLL03')

  0%|                                                                                                                                                                            | 0/14041 [00:00<?, ?it/s]

['train.word.bmes', 'test.word.bmes', 'dev.word.bmes']
10099


  1%|█                                                                                                                                                                  | 22/3453 [00:00<00:16, 211.99it/s]

parsing error count :  0 14041
12539


  1%|▉                                                                                                                                                                  | 19/3250 [00:00<00:17, 188.83it/s]

parsing error count :  0 3453
14961


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3250/3250 [00:13<00:00, 238.33it/s]

parsing error count :  0 3250
